In [6]:
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [7]:
from selenium import webdriver
from chromedriver_py import binary_path # this will get you the path variable
from selenium.webdriver.common.keys import Keys

# Кликнуть по первой кнопке из списка, открыв ссылку в новой вкладке
#svc = webdriver.ChromeService(executable_path=binary_path)

In [8]:
base_url = 'https://ria.ru/category_pogoda/'

In [9]:
#pip install bs4

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

In [11]:
df = pd.DataFrame(columns=['text'])

In [14]:
driver = webdriver.Chrome()

# Задержка перед открытием страницы
time.sleep(15)

# Открываем базовый URL
base_url = "https://ria.ru/category_pogoda/"
driver.get(base_url)

# Задержка перед поиском элементов
time.sleep(5)
i = 0
cur_iter = 0
while i <= 10:

    # Находим все ссылки на новости
    link_elements = driver.find_elements(By.XPATH, "//a[@class='list-item__title color-font-hover-only']")

    # Проходимся по всем найденным элементам
    for link_element in link_elements[cur_iter:]:
        # Получаем ссылку из элемента
        link = link_element.get_attribute("href")
        # Открываем ссылку
        #driver.get(link)
        driver.execute_script("window.open(arguments[0], '_blank');", link)
        # Задержка перед обработкой страницы
        time.sleep(5)
        # Получаем HTML-код страницы
        html_content = driver.page_source
        # Создаем объект BeautifulSoup для парсинга HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        article_text_elements = soup.find_all('div', class_='article__text')
        # Объявляем переменную для хранения полного текста
        full_text = ""

        # Обходим все найденные элементы и собираем текст вместе
        for element in article_text_elements:
            text = element.get_text(strip=True)
            full_text += text + "\n"

        # Выводим полный текст
        print(full_text)
        df.loc[len(df)] = [full_text]
        all_handles = driver.window_handles
        driver.switch_to.window(all_handles[-1])
        time.sleep(3)
        driver.close()
        time.sleep(3)
                # Переключаемся обратно на базовую вкладку
        base_window_handle = all_handles[0]
        driver.switch_to.window(base_window_handle)

    cur_iter = len(df)
    df.to_csv('weather_df.csv')
    driver.execute_script("window.scrollBy(0, 1000);")
    button = driver.find_element(By.CSS_SELECTOR, "div.list-more.color-btn-second-hover")
    i+=1


In [6]:
def iterations(driver, link_elements, cur_iter, df):
    for link_element in link_elements[cur_iter:]:
        # Получаем ссылку из элемента
        link = link_element.get_attribute("href")
        # Открываем ссылку
        #driver.get(link)
        driver.execute_script("window.open(arguments[0], '_blank');", link)
        # Задержка перед обработкой страницы
        time.sleep(5)
        # Получаем HTML-код страницы
        html_content = driver.page_source
        # Создаем объект BeautifulSoup для парсинга HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        article_text_elements = soup.find_all('div', class_='article__text')
        # Объявляем переменную для хранения полного текста
        full_text = ""

        # Обходим все найденные элементы и собираем текст вместе
        for element in article_text_elements:
            text = element.get_text(strip=True)
            full_text += text + "\n"
            
        if full_text != '':
            df.loc[len(df)] = [full_text]
    cur_iter += len(link_elements)
    button = driver.find_element(By.CSS_SELECTOR, "div.list-more.color-btn-second-hover")
    iterations(driver, link_elements, cur_iter, df)